In [ ]:
!pip install -q -U  datasets tqdm peft  accelerate transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompa

In [ ]:
!pip install duckpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
from duckpy import Client

client = Client()

results = client.search("jokes")

# Prints first result title
print(results[0].title)

# Prints first result URL
print(results[0].url)

# Prints first result description
print(results[0].description)

IndexError: list index out of range

In [ ]:
results

[]

In [ ]:
!pip install -q dataclass_csv

In [ ]:
import os
import warnings
import torch
from tqdm import tqdm
from accelerate import Accelerator
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_int8_training,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    default_data_collator,
    BitsAndBytesConfig
)
from dataclasses import dataclass, field
from torch.utils.data import Dataset, DataLoader
from enum import Enum
from datasets import load_dataset,DatasetDict
from typing import Union , Dict,Optional,Any,List

In [ ]:
def advanced_data_loader(input: Union[str, Dict[str, str]], format: Optional[str] = None, split_ratios: Optional[Dict[str, float]] = None) -> Optional[DatasetDict]:
    """
    Loads a dataset from a given input path or dictionary specifying file paths and splits it.

    :param input: A string representing the dataset name or directory, or a dictionary containing file paths.
    :param format: The format of the dataset if loading from a file (e.g., 'csv' or 'json').
    :param split_ratios: A dictionary with keys 'train', 'test', and 'eval' containing split ratios.
    :return: A loaded and split dataset or None in case of failure.
    """
    if split_ratios is None:
        split_ratios = {'train': 0.8, 'test': 0.1, 'eval': 0.1}

    try:
        # Load the dataset
        if isinstance(input, dict) and format in ['csv', 'json']:
            dataset = load_dataset(format, data_files=input)
        elif isinstance(input, str) and format == 'text':
            dataset = load_dataset(format, data_dir=input)
        elif isinstance(input, str) and format is None:
            dataset = load_dataset(input)
        else:
            warnings.warn("Invalid input or format. Please provide a valid dataset name, directory, or file paths.")
            return None
    except FileNotFoundError as e:
        warnings.warn(str(e))
        return None

    # Split the dataset
    if dataset:
        split_dataset = dataset['train'].train_test_split(test_size=split_ratios['test'] + split_ratios['eval'])
        test_eval_dataset = split_dataset['test'].train_test_split(test_size=split_ratios['eval'] / (split_ratios['test'] + split_ratios['eval']))
        dataset = DatasetDict({
            'train': split_dataset['train'],
            'test': test_eval_dataset['train'],
            'eval': test_eval_dataset['test']
        })

    print("Splits: ", dataset.keys())
    print("Columns: ", {split: dataset[split].column_names for split in dataset.keys()})
    return dataset

In [ ]:
def create_tokenizer(tokenizer_name_or_path: str = 'gpt2') -> AutoTokenizer:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    if tokenizer.bos_token_id is None:
        tokenizer.bos_token_id = tokenizer.pad_token_id
    if tokenizer.eos_token_id is None:
        tokenizer.eos_token_id = tokenizer.pad_token_id
    if tokenizer.unk_token_id is None:
        tokenizer.unk_token_id = tokenizer.pad_token_id
    if tokenizer.sep_token_id is None:
        tokenizer.sep_token_id = tokenizer.pad_token_id
    if tokenizer.cls_token_id is None:
        tokenizer.cls_token_id = tokenizer.pad_token_id
    if tokenizer.mask_token_id is None:
        tokenizer.mask_token_id = tokenizer.pad_token_id
    return tokenizer


In [ ]:

input1={'train':'/content/drive/MyDrive/Adversarial_chatbot_dataset _test.csv'}
dataset=advanced_data_loader(input=input1,format='csv')

Splits:  dict_keys(['train', 'test', 'eval'])
Columns:  {'train': ['id', 'input', 'output'], 'test': ['id', 'input', 'output'], 'eval': ['id', 'input', 'output']}


In [ ]:
def preprocess_function(examples):
  """
  Preprocess the dataset.
  """
  batch_size = len(examples[TEXT_COLUMN])
  inputs = [f"{TEXT_COLUMN } : {x} Label : " for x in examples[TEXT_COLUMN ]]
  targets = [str(x) for x in examples[LABEL_COLUMN]]
  model_inputs = tokenizer(inputs)
  labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
  for i in range(batch_size):
      sample_input_ids = model_inputs["input_ids"][i]
      label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
      model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
      labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
      model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
  for i in range(batch_size):
      sample_input_ids = model_inputs["input_ids"][i]
      label_input_ids = labels["input_ids"][i]
      model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
          MAX_LENGTH - len(sample_input_ids)
      ) + sample_input_ids
      model_inputs["attention_mask"][i] = [0] * (MAX_LENGTH - len(sample_input_ids)) + model_inputs[
          "attention_mask"
      ][i]
      labels["input_ids"][i] = [-100] * (MAX_LENGTH - len(sample_input_ids)) + label_input_ids
      model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:MAX_LENGTH])
      model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:MAX_LENGTH])
      labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:MAX_LENGTH])
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

def test_preprocess_function(examples):
  batch_size = len(examples[TEXT_COLUMN])
  inputs = [f"{TEXT_COLUMN} : {x} Label : " for x in examples[TEXT_COLUMN]]
  model_inputs = tokenizer(inputs)
  # print(model_inputs)
  for i in range(batch_size):
      sample_input_ids = model_inputs["input_ids"][i]
      model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
          MAX_LENGTH - len(sample_input_ids)
      ) + sample_input_ids
      model_inputs["attention_mask"][i] = [0] * (MAX_LENGTH - len(sample_input_ids)) + model_inputs[
          "attention_mask"
      ][i]
      model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:MAX_LENGTH])
      model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:MAX_LENGTH])
  return model_inputs

In [ ]:
model_name_or_path='gpt2'
TEXT_COLUMN='output'
LABEL_COLUMN='input'
MAX_LENGTH=512
BATCH_SIZE=16
tokenizer_name_or_path='gpt2'
tokenizer=create_tokenizer(tokenizer_name_or_path)
COLUMNS_NAME=None
Columns=None
COLUMNS=dataset['train'].column_names

In [ ]:
accelerator = Accelerator()

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 528
    })
    test: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 66
    })
    eval: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 66
    })
})

In [ ]:

with accelerator.main_process_first():
  processed_datasets = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=True,
            desc="Running tokenizer on dataset",
)
train_dataset = processed_datasets["train"]
with accelerator.main_process_first():
  processed_datasets = dataset.map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["test"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
  )
test_dataset = processed_datasets["test"]
with accelerator.main_process_first():
  processed_datasets = dataset.map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["test"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
  )
eval_dataset = processed_datasets["eval"]
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset, shuffle=True,collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)


Running tokenizer on dataset:   0%|          | 0/528 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/66 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/66 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/528 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/66 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/66 [00:00<?, ? examples/s]

In [ ]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 528
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 66
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 66
    })
})

In [ ]:
processed_datasets['train'][0]

In [ ]:

with accelerator.main_process_first():
  processed_datasets = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=True,
            desc="Running tokenizer on dataset",
)
train_dataset = processed_datasets["train"]
with accelerator.main_process_first():
  processed_datasets = dataset.map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["test"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
  )
test_dataset = processed_datasets["test"]
with accelerator.main_process_first():
  processed_datasets = dataset.map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["test"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
  )
eval_dataset = processed_datasets["eval"]
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset, shuffle=True,collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
